In [ ]:
#importing the libraries

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from keras.applications.resnet import ResNet50
from keras.applications.resnet import preprocess_input

from keras import models, Sequential #model
from keras.layers import Dense, Flatten #layers

## The coco dataset

In [ ]:
####################### Load the COCO dataset ###################################
path = "C:\\Users\\Samruddhi\\Desktop\\Neural Networks\\datasets"  #dataset should be downloaded here

dataset, info = tfds.load('coco', split='train', with_info=True, data_dir = path, as_supervised=True)
    #as_supervised = want labels as well?

# Total number of examples in the dataset
total = info.splits['train'].num_examples



########################### split into training, validation and test sets ################
# proportions
num_train = int(0.8 * total) # 80%
num_val = int(0.1 * total)   # 10%
num_test = int(0.1 * total)  # 10%

# Shuffle the dataset
dataset = dataset.shuffle(total, seed=42, reshuffle_each_iteration=False)

# Split 
training_set = dataset.take(num_train)
val_set = dataset.skip(num_train).take(num_val)
test_set = dataset.skip(num_train + num_val).take(num_test)


########################### data pre-processing ####################################
def preprocess_image(image, label):
    image = preprocess_input(image)
    return image, label

training_set = training_set.map(preprocess_image)
val_set = val_set.map(preprocess_image)
test_set = test_set.map(preprocess_image)


########################### divide into batches #########################################
batch_size = 32
training_set = training_set.batch(batch_size)
val_set = val_set.batch(batch_size)
test_set = test_set.batch(batch_size)

## The model

In [ ]:
# pre-trained ResNet50 model
conv_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


# custom model 
model = Sequential([
    conv_base,
    Flatten(),
    Dense(100, activation = "relu"),
    Dense(80, activation = "softmax") #output layer
])


## Training

In [ ]:
#freeze the convo base
conv_base.trainable = False

#compile
model.compile(optimizer = 'adam', loss = keras.losses.SparseCategoricalCrossentropy(), metrics = ['accuracy'])

#training and validation
model.fit_generator(training_set, validation_data= val_set, epochs = 15)

#save the trained model
model.save('trained_model.h5')

## Testing

In [ ]:
################ testing ########################
# Load the trained model
model = models.load_model('trained_model.h5')

#make the predictions
predictions = model.predict(test_set)

# The predicted labels
predicted_labels = tf.argmax(predictions, axis=1).numpy()

# The true labels
true_labels = tf.concat([y for x, y in test_set], axis=0).numpy()

# Calculate accuracy
accuracy = (predicted_labels == true_labels).mean()
print("Accuracy:", accuracy)